In [1]:
#1. Generate dataset
#2. Train the classifier and save it
#3. Detect the face and named it if it is already stored in our dataset

# Generate dataset

In [2]:
import cv2

In [3]:
def generate_dataset() :
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if faces is () :
            return None
        for(x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
        return cropped_face
    cap = cv2.VideoCapture(0)
    id = 3
    img_id = 0
    
    while True :
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "data/user." + str(id) + "." + str(img_id) + ".jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            
            cv2.imshow("Cropped face", face)
            if cv2.waitKey(1) == 13 or int(img_id) == 100:
                break
    cap.release()
    cv2.destroyAllWindows()
    
generate_dataset()             
            

<>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\MINH\AppData\Local\Temp\ipykernel_31180\2887593708.py:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is () :


# Train classifier and save it

In [4]:
import numpy as np
from PIL import Image
import os
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pickle
import joblib

In [5]:
def train_classifier(data_dir) :
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []
    
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        faces.append(imageNp)
        ids.append(id)
    ids = np.array(ids)
    #feature extraction
    pca = PCA(n_components=1)
    image_pca = []
    for img_vector in faces:
        pca.fit(img_vector)
        img_pca = pca.transform(img_vector)
        image_pca.append(img_pca.flatten())
    #train classifier and save it
    clf = SVC(kernel = 'rbf', class_weight='balanced', probability=True)
    clf.fit(image_pca, ids)
    joblib.dump(clf, 'classifier.pkl')
    
train_classifier("data")

ValueError: invalid literal for int() with base 10: '2154810087-herman'

# Detect the face and named it if it is already stored in our dataset

In [ ]:
import cv2
import numpy as np
from PIL import Image
import os
import joblib
from sklearn.decomposition import PCA
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf, pca) :
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)
    
    coords = []
    for(x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        img_cropped = cv2.resize(gray_image[y:y+h,x:x+w], (200, 200))
        pca.fit(img_cropped)
        img_pca = pca.transform(img_cropped).flatten()
        pred = clf.predict([img_pca])
        proba = clf.predict_proba([img_pca])
        max_proba = max(proba.flatten())
        print(max_proba)
        if int(max_proba * 100) >= 82:
            cv2.putText(img, str(pred), (x, y-5), cv2.FONT_HERSHEY_COMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else :
            cv2.putText(img, "UNKNOWN", (x, y-5), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
        
        coords = [x, y, w, h]
    return coords

def recognize(img, clf, faceScascade, pca) :
    coords = draw_boundary(img, faceCascade, 1.3, 5, (255, 255, 255), "Face", clf, pca)
    return img

faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf = joblib.load("classifier.pkl")
pca = PCA(n_components=1)
video_capture = cv2.VideoCapture(0)

while True:
    ret, img = video_capture.read()
    img = recognize(img, clf, faceCascade, pca)
    cv2.imshow("face detection", img)
    
    if cv2.waitKey(1) == 13 :
        break
video_capture.release()
cv2.destroyAllWindows()